# SEO Notebook

In [ ]:
"""A framework for conducting SEO investigations, using Google Spreadsheets for I/O."""

In [ ]:
import notebook_finder    #Allows .ipynb files to be loaded like filename.py modules
import goodsheet          #Handles OAuth login for all relevant Google services
from imp import reload    #Keeps in-memory functions reflecting your pipulate.ipynb edits
import shelve

In [ ]:
cache_html = True
google_sheet_name = 'Lookups'
google_sheet_tab  = 'Sheet1'

In [ ]:
if __name__ == '__main__':
    import pipulate
    reload(pipulate)
    import pipulate #intentional
    worksheet = goodsheet.connect.open(google_sheet_name).worksheet(google_sheet_tab)
    pipulate_funcs = [x for x in dir(pipulate) if x[0] is not '_']
    func_dict = {x.lower():eval('pipulate.%s' % x) for x in pipulate_funcs}
    try:
        end_range = worksheet.get_addr_int(worksheet.row_count, worksheet.col_count)
        cell_range = worksheet.range('A1:%s' % end_range)
        col_count = worksheet.col_count
    except:
        print("Cannot reach Google Sheets.")
        raise SystemExit()
    col_names = []
    row_dict = {}
    #if cache_html == True:
    #    urls = shelve.open('urls.db')
    print("Pipulating row: ", end="")
    for i, acell in enumerate(cell_range):
        row, col, val = acell.row, acell.col, acell.value
        if (col)%col_count == 0:
            print(row, end=", ")
        if row == 1:
            col_names.append(val.lower())
        else:
            row_dict[col_names[col-1]] = val
            if col%col_count == 0:
                # Reached end of row, let's Pipulate!
                if 'url' in row_dict:
                    #raise SystemExit()
                    pass
                for key, val in row_dict.items():
                    if not val:
                        if key in [x.lower() for x in dir(pipulate) if x[0] is not '_']:
                            success_code, new_text = func_dict[key](**row_dict)
                            cell_address = (row*col_count) - col_count + col_names.index(key)
                            cell_range[cell_address].value = new_text
                row_dict = {}
    print("Done, updating spreadsheet...")
    try:
        worksheet.update_cells(cell_range)
        print("Updated!")
    except:
        print("Update failed.")